In [31]:
import os
import pickle
import re
from pathlib import Path
import pandas as pd
from tsfresh import extract_features

In [32]:
CURRENT_DIR = Path.cwd()
PARENT_DIR = CURRENT_DIR.parent
DATA_DIR = PARENT_DIR / "P02_data" / "T02_af"
print(CURRENT_DIR)
print(DATA_DIR)

c:\Users\admin\Coding\research\weld-ml\src\P03_data_preprocess
c:\Users\admin\Coding\research\weld-ml\src\P02_data\T02_af


In [33]:
pickle_filepath = DATA_DIR / "S17_af_extract.pkl"
with open(pickle_filepath, "rb") as handle:
    data = pickle.load(handle)
data = data["data"]

In [34]:
num_samples = 3
data_arr = []
for sample_no in data["sample_no"].unique()[:num_samples]:
    af = data.loc[data["sample_no"] == sample_no, "af_dwell"].values[0]
    _fz = af[["Fz", "sample_no"]].copy()
    _fz.index = _fz.index - _fz.index[0]  # Reset time index to start from 0
    data_arr.append(_fz)

df_fz = pd.concat(data_arr)
df_fz =df_fz.reset_index()


In [35]:
_ext_dwell = extract_features(df_fz, column_id="sample_no", column_sort="Time", column_value="Fz")

Feature Extraction: 100%|██████████| 3/3 [00:07<00:00,  2.43s/it]


In [36]:
_ext_dwell

,Fz__variance_larger_than_standard_deviation,Fz__has_duplicate_max,Fz__has_duplicate_min,Fz__has_duplicate,Fz__sum_values,Fz__abs_energy,Fz__mean_abs_change,Fz__mean_change,Fz__mean_second_derivative_central,Fz__median,...,Fz__fourier_entropy__bins_5,Fz__fourier_entropy__bins_10,Fz__fourier_entropy__bins_100,Fz__permutation_entropy__dimension_3__tau_1,Fz__permutation_entropy__dimension_4__tau_1,Fz__permutation_entropy__dimension_5__tau_1,Fz__permutation_entropy__dimension_6__tau_1,Fz__permutation_entropy__dimension_7__tau_1,Fz__query_similarity_count__query_None__threshold_0.0,Fz__mean_n_absolute_max__number_of_maxima_7
1,0.0,0.0,0.0,1.0,2292.31420,5263.882785,0.067820,-0.000466,-0.000017,2.27687,...,0.136002,0.181214,0.380783,1.692953,2.579183,3.470798,4.453641,5.302442,NaN,2.840246
2,0.0,0.0,0.0,1.0,3925.10689,10278.990476,0.081735,-0.000060,0.000039,2.61169,...,0.090729,0.136002,0.226363,1.706271,2.499941,3.162247,3.848210,4.512462,NaN,3.025519
3,0.0,0.0,0.0,1.0,5147.68669,13260.798765,0.079072,-0.000198,0.000031,2.56648,...,0.090729,0.136002,0.226363,1.707461,2.513931,3.306369,4.182854,5.033741,NaN,3.008877


In [37]:
# Some features have NaN.
from sklearn.impute import SimpleImputer

# Drop columns which contains all NaN.
_ext_dwell_drop = _ext_dwell.dropna(axis=1, how="any")

imputer = SimpleImputer(strategy="mean")
exIm = imputer.fit_transform(_ext_dwell_drop)
ext_dwell = pd.DataFrame(exIm, columns=_ext_dwell_drop.columns, index=_ext_dwell_drop.index)